In [1]:
from carro_scraper import WebPage, wait_for_element

In [2]:
import undetected_chromedriver as uc
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [30]:
def search_product(product_id):


    options = webdriver.ChromeOptions() 
    options.add_argument('--start-maximized')
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-dve-shm-uage')
    
    driver = uc.Chrome(options=options)

    hd_url = 'https://www.homedepot.com/'

    driver.get(hd_url)

    search_box_xpath = '//*[@id="typeahead-search-field-input"]'

    search_box = WebPage.wait_for_element(driver, search_box_xpath)

    search_box.send_keys(product_id)
    search_box.send_keys(Keys.RETURN) 
    time.sleep(5) 

    product = WebPage.from_web_element(driver)

    driver.close()

    return product


In [25]:
carro_products = pd.read_excel("HD SKU前台数据_080524.xlsx")
carro_products['<ID>'] = carro_products['<ID>'].astype(str)

In [26]:
product_id = carro_products['<ID>'][0]
product_id

'321644000'

In [31]:
search_product(product_id)

could not detect version_main.therefore, we are assuming it is chrome 108 or higher


KeyboardInterrupt: 

In [8]:
from concurrent.futures import ThreadPoolExecutor, as_completed

In [15]:
def fetch_details_for_all_rows(df):
    with ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(search_product, row['<ID>']): index for index, row in df.iterrows()}
        for future in as_completed(future_to_url):
            index = future_to_url[future]

            try:
                detail_info = future.result()
                df.at[index, 'Rating'] = detail_info.rating
                df.at[index, '2024/8/5 MAP'] = detail_info.price
                df.at[index, 'Status(Available/Unavailable/Out of Stock)'] = detail_info.status

            except Exception as e:
                print(e)

    return df

In [21]:
df = carro_products.head(2)

In [18]:
df.to_excel("test.xlsx")

In [22]:
for index, row in df.iterrows():
    product = search_product(row['<ID>'])
    df.at[index, 'Rating'] = product.rating
    df.at[index, '2024/8/5 MAP'] = product.price
    df.at[index, 'Status(Available/Unavailable/Out of Stock)'] = product.status

C:\Users\cs\AppData\Local\Temp\ipykernel_1448\238864874.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '4 out of 5' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Rating'] = product.rating
C:\Users\cs\AppData\Local\Temp\ipykernel_1448\238864874.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '$199.00' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, '2024/8/5 MAP'] = product.price
C:\Users\cs\AppData\Local\Temp\ipykernel_1448\238864874.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Not Available in California' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Statu

In [23]:
df

,MFG Model #,<ID>,<Object Type Name>,<Parent ID>,Rating,2024/8/5 MAP,Inventory,Status(Available/Unavailable/Out of Stock),Product Name (120)
0,HC523P-N10-W1-1-FM,321644000,Super SKU Item,321643941,4 out of 5,$199.00,NaN,Not Available in California,Modena 52 in. Indoor White 10-Speed DC Motor F...
1,HC523P-N10-G2-1-FM,321646581,Super SKU Item,321643941,4 out of 5,$199.00,NaN,Not Available in California,Modena 52 in. Indoor Gold 10-Speed DC Motor Fl...
